In [1]:
import numpy as np
import pandas as pd
from numba import jit
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from scipy.spatial import distance_matrix
from cvxopt import matrix, solvers
import time

# Auxiliary function

In [2]:
def discriminate(x):
    if x>0:
        return 1
    else:
        return 0
discriminate_vec = np.vectorize(discriminate)

# Gaussian kernel SVM on given matrices:

In [3]:
# Hyperparameters:
lambd_SVM = 0.0001
sigma = 0.1

# Main code:
export_header = True
for dataset in range(3):
    # Load dataset:
    Xtr = pd.read_csv("data/Xtr"+str(dataset)+"_mat100.csv", sep=" ", header=None).to_numpy()
    Xte = pd.read_csv("data/Xte"+str(dataset)+"_mat100.csv", sep=" ", header=None).to_numpy()
    Ytr = pd.read_csv("data/Ytr"+str(dataset)+".csv").to_numpy()[:,1]
    Ytr = Ytr.reshape((Ytr.shape[0],1))
    Ytr_reshaped = 2*Ytr.reshape(Ytr.shape[0])-1
    # Build Gram matrix:
    d_matrix_train = distance_matrix(Xtr, Xtr)
    K_train = np.exp(-d_matrix_train*d_matrix_train/(2*sigma*sigma))
    # Set problem for QP solver:
    ## Number of samples:
    n  = K_train.shape[0]
    ## Initial guess for mu:
    x0 = np.zeros(2*n)
    ## Matrix P:
    P = matrix((1/(2*lambd_SVM))*np.diag(Ytr_reshaped)@K_train@np.diag(Ytr_reshaped))
    ## Vector Q:
    q = -np.ones(n)
    q = matrix(q)
    ## Matrix G:
    G = np.zeros((2*n,n))
    G[:n] = -np.eye(n)
    G[n:] = np.eye(n)
    G = matrix(G)
    ## Vector h:
    h = np.zeros(2*n)
    h[n:]+= 1/n
    h = matrix(h)
    ## Call QP solver on dual SVM problem:
    resu_SVM = solvers.qp(P, q, G, h)
    mu = np.array(resu_SVM["x"])
    ## Recover primal solution:
    alpha = np.diag(Ytr_reshaped)@mu/(2*lambd_SVM)
    ## Build train prediction
    y_pred_SVM_raw = K_train@alpha
    y_pred_SVM = discriminate_vec(y_pred_SVM_raw)
    ## Show training accuracy:
    print('Training Accuracy for training set '+str(dataset)+' :',accuracy_score(y_pred_SVM, Ytr))
    # Test:
    ## Build Gram matrix:
    d_matrix_test = distance_matrix(Xte, Xtr)
    K_test = np.exp(-d_matrix_test*d_matrix_test/(2*sigma*sigma))
    ## Build test prediction:
    y_pred_SVM_test_raw = K_test@alpha
    y_pred_SVM_test = discriminate_vec(y_pred_SVM_test_raw)
    ## Build dataframe and save as .csv:
    start_index = dataset*1000
    dico = {"Id": range(start_index, start_index+Xte.shape[0]), "Bound": y_pred_SVM_test.reshape(Xte.shape[0]) }
    pred_SVM_test = pd.DataFrame(dico)
    pred_SVM_test.to_csv("output_Gaussian_SVM.csv", index=False, sep=",", header=export_header, mode='a')
    export_header = False

     pcost       dcost       gap    pres   dres
 0: -7.3319e-01 -1.7373e+00  4e+03  6e+01  4e-14
 1: -7.3315e-01 -1.7321e+00  7e+01  1e+00  5e-14
 2: -7.0844e-01 -1.5748e+00  1e+01  2e-01  4e-14
 3: -5.4846e-01 -1.3311e+00  2e+00  3e-02  3e-14
 4: -4.4610e-01 -8.2229e-01  4e-01  3e-04  2e-14
 5: -4.7319e-01 -5.3336e-01  6e-02  5e-05  1e-14
 6: -4.8540e-01 -4.9402e-01  9e-03  3e-06  2e-14
 7: -4.8773e-01 -4.8846e-01  7e-04  1e-07  2e-14
 8: -4.8796e-01 -4.8801e-01  5e-05  6e-09  2e-14
 9: -4.8797e-01 -4.8798e-01  2e-06  1e-10  2e-14
10: -4.8798e-01 -4.8798e-01  8e-08  2e-12  2e-14
Optimal solution found.
Training Accuracy for training set 0 : 0.93
     pcost       dcost       gap    pres   dres
 0: -7.4903e-01 -1.7501e+00  4e+03  6e+01  3e-14
 1: -7.4900e-01 -1.7451e+00  5e+01  8e-01  4e-14
 2: -7.1824e-01 -1.5457e+00  9e+00  1e-01  3e-14
 3: -4.8773e-01 -1.3535e+00  2e+00  1e-02  2e-14
 4: -4.6528e-01 -6.6498e-01  2e-01  4e-04  1e-14
 5: -4.9463e-01 -5.2219e-01  3e-02  4e-05  1e-14
 6:

# More specific kernels:

In [4]:
def spectrum_kernel(X, Y, **kwargs):
    k = kwargs["k"]
    m1 = X.shape[0]
    m2 = Y.shape[0]
    list_of_dictionaries_X = []
    for chaine in X["seq"]:
        dico = {}
        for i in range(len(chaine)-k+1):
            motif = chaine[i:i+k]
            if not(motif in dico.keys()):
                dico[motif]=1
            else:
                dico[motif]+=1
        list_of_dictionaries_X.append(dico)
    list_of_dictionaries_Y = []
    for chaine in Y["seq"]:
        dico = {}
        for i in range(len(chaine)-k+1):
            motif = chaine[i:i+k]
            if not(motif in dico.keys()):
                dico[motif]=1
            else:
                dico[motif]+=1
        list_of_dictionaries_Y.append(dico)
    mat = np.zeros((m1,m2))
    for i in range(m1):
        for j in range(m2):
            for key in set(list_of_dictionaries_X[i].keys())&set(list_of_dictionaries_Y[j].keys()):
                mat[i][j]+= list_of_dictionaries_X[i][key]*list_of_dictionaries_Y[j][key]
    return mat

In [5]:
def build_1neighborhood(chain):
    chain = list(chain)
    lili = []
    for i in range(len(chain)):
        temp = chain.copy()
        for char in ['A', 'G', 'C', 'T']:
            temp[i]=char
            lili.append("".join(temp))
    return lili

    

In [6]:
len(build_1neighborhood("GTGCCGACGCAGCGGTGTTGCACCTCCCT"))

116

In [7]:
def mismatch_kernel(X, Y, **kwargs):
    k = kwargs["k"]
    m1 = X.shape[0]
    m2 = Y.shape[0]
    list_of_dictionaries_X = []
    for chaine in X["seq"]:
        dico = {}
        for i in range(len(chaine)-k+1):
            motif = chaine[i:i+k]
            for neighbor in build_1neighborhood(motif):
                if not(neighbor in dico.keys()):
                    dico[neighbor]=1
                else:
                    dico[neighbor]+=1
        list_of_dictionaries_X.append(dico)
    list_of_dictionaries_Y = []
    for chaine in Y["seq"]:
        dico = {}
        for i in range(len(chaine)-k+1):
            motif = chaine[i:i+k]
            for neighbor in build_1neighborhood(motif):
                if not(neighbor in dico.keys()):
                    dico[neighbor]=1
                else:
                    dico[neighbor]+=1
        list_of_dictionaries_Y.append(dico)
    mat = np.zeros((m1,m2))
    for i in range(m1):
        for j in range(m2):
            for key in set(list_of_dictionaries_X[i].keys())&set(list_of_dictionaries_Y[j].keys()):
                mat[i][j]+= list_of_dictionaries_X[i][key]*list_of_dictionaries_Y[j][key]
    return mat

# Generic Kernel SVM on given matrices
## Training on 75% of the training set and testing on the remaining 25\%.


In [8]:
def run_SVM_validation(K, name, prop=0.75, lambd_SVM = 0.001, **kwargs):
    start_time = time.time()
    # Main code:
    export_header = True
    for dataset in range(3):
        start_step = time.time()
        # Load dataset:
        X = pd.read_csv("data/Xtr"+str(dataset)+".csv", sep=",")
        slice_index = int(prop*X.shape[0])
        Xtr = X.head(slice_index)
        Xte = X.tail(X.shape[0]-slice_index)
        Y = pd.read_csv("data/Ytr"+str(dataset)+".csv")
        Ytr = Y.head(slice_index).to_numpy()[:,1]
        Ytr = Ytr.reshape((Ytr.shape[0],1))
        Ytr_reshaped = 2*Ytr.reshape(Ytr.shape[0])-1
        Yte = Y.tail(Y.shape[0]-slice_index).to_numpy()[:,1]
        Yte = Yte.reshape((Yte.shape[0],1))
        # Build Gram matrix:
        K_train = K(Xtr, Xtr, **kwargs)
        print("K_train successfully built")
        # Set problem for QP solver:
        ## Number of samples:
        n  = K_train.shape[0]
        ## Initial guess for mu:
        x0 = np.zeros(2*n)
        ## Matrix P:
        P = matrix((1/(2*lambd_SVM))*np.diag(Ytr_reshaped)@K_train@np.diag(Ytr_reshaped))
        ## Vector Q:
        q = -np.ones(n)
        q = matrix(q)
        ## Matrix G:
        G = np.zeros((2*n,n))
        G[:n] = -np.eye(n)
        G[n:] = np.eye(n)
        G = matrix(G)
        ## Vector h:
        h = np.zeros(2*n)
        h[n:]+= 1/n
        h = matrix(h)
        ## Call QP solver on dual SVM problem:
        resu_SVM = solvers.qp(P, q, G, h)
        mu = np.array(resu_SVM["x"])
        ## Recover primal solution:
        alpha = np.diag(Ytr_reshaped)@mu/(2*lambd_SVM)
        ## Build train prediction
        y_pred_SVM_raw = K_train@alpha
        y_pred_SVM = discriminate_vec(y_pred_SVM_raw)
        ## Show training accuracy:
        print('Training Accuracy for training set '+str(dataset)+' :',accuracy_score(y_pred_SVM, Ytr))
        # Test:
        ## Build Gram matrix:
        K_test = K(Xte, Xtr, **kwargs)
        print("K_test successfully built")
        ## Build test prediction:
        y_pred_SVM_test_raw = K_test@alpha
        y_pred_SVM_test = discriminate_vec(y_pred_SVM_test_raw)
        ## Show test accuracy:
        print('Testing Accuracy for training set '+str(dataset)+' :',accuracy_score(y_pred_SVM_test, Yte))
        ## Build dataframe and save as .csv:
        start_index = dataset*1000
        dico = {"Id": range(start_index, start_index+Xte.shape[0]), "Bound": y_pred_SVM_test.reshape(Xte.shape[0]) }
        pred_SVM_test = pd.DataFrame(dico)
        pred_SVM_test.to_csv(name+".csv", index=False, sep=",", header=export_header, mode='a')
        export_header = False
        end_step = time.time()
        print("time on dataset {}: {}".format(dataset,np.round(end_step-start_step,2)))
    end_time = time.time()
    print("total time: {}".format(np.round(end_time-start_time,2)))

In [9]:
run_SVM_validation(spectrum_kernel, "validation_spectrum_kernel_SVM", lambd_SVM = 0.01, **{"k":10})

K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.5262e-01 -1.1532e+00  3e+03  5e+01  7e-16
 1: -1.5262e-01 -1.1524e+00  3e+01  6e-01  4e-16
 2: -1.5232e-01 -1.0822e+00  2e+00  2e-02  4e-16
 3: -1.4807e-01 -4.9780e-01  4e-01  1e-03  4e-16
 4: -1.5130e-01 -1.8768e-01  4e-02  1e-04  4e-16
 5: -1.5179e-01 -1.5539e-01  4e-03  9e-06  3e-16
 6: -1.5185e-01 -1.5210e-01  2e-04  6e-07  3e-16
 7: -1.5186e-01 -1.5186e-01  7e-06  1e-08  3e-16
 8: -1.5186e-01 -1.5186e-01  2e-07  3e-10  3e-16
 9: -1.5186e-01 -1.5186e-01  7e-09  3e-12  3e-16
Optimal solution found.
Training Accuracy for training set 0 : 1.0
K_test successfully built
Testing Accuracy for training set 0 : 0.604
time on dataset 0: 53.03
K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.5568e-01 -1.1561e+00  3e+03  5e+01  7e-16
 1: -1.5568e-01 -1.1553e+00  3e+01  6e-01  4e-16
 2: -1.5541e-01 -1.0861e+00  2e+00  1e-02  4e-16
 3: -1.5314e-01 -3.5707e-01  2e-01  5e-18  3e-16
 4

In [10]:
run_SVM_validation(mismatch_kernel, "validation_mismatch_kernel_SVM", lambd_SVM = 0.01, **{"k":10})

K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.1606e-03 -1.0012e+00  3e+03  5e+01  1e-15
 1: -1.1606e-03 -9.9985e-01  3e+01  5e-01  2e-15
 2: -1.1436e-03 -8.8078e-01  1e+00  6e-03  2e-15
 3: -2.4543e-04 -6.2243e-02  7e-02  3e-04  2e-15
 4: -6.7947e-04 -1.0474e-02  1e-02  2e-05  2e-15
 5: -1.1329e-03 -1.5485e-03  4e-04  3e-07  2e-15
 6: -1.1549e-03 -1.1925e-03  4e-05  2e-08  2e-15
 7: -1.1559e-03 -1.1601e-03  4e-06  2e-09  2e-15
 8: -1.1560e-03 -1.1562e-03  1e-07  6e-11  2e-15
 9: -1.1560e-03 -1.1560e-03  3e-09  1e-12  2e-15
Optimal solution found.
Training Accuracy for training set 0 : 1.0
K_test successfully built
Testing Accuracy for training set 0 : 0.624
time on dataset 0: 1420.74
K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.1707e-03 -1.0012e+00  3e+03  5e+01  1e-15
 1: -1.1707e-03 -9.9986e-01  3e+01  5e-01  2e-15
 2: -1.1535e-03 -8.8077e-01  1e+00  6e-03  2e-15
 3: -2.0806e-04 -4.6305e-02  5e-02  2e-04  2e-15


# Generic Kernel SVM on given matrices
## Learning on the full training set and producing predictions on the test set




In [11]:
def run_SVM(K, name, lambd_SVM = 0.001, **kwargs):
    start_time = time.time()
    # Main code:
    export_header = True
    for dataset in range(3):
        start_step = time.time()
        # Load dataset:
        Xtr = pd.read_csv("data/Xtr"+str(dataset)+".csv", sep=",")
        Xte = pd.read_csv("data/Xte"+str(dataset)+".csv", sep=",")
        Ytr = pd.read_csv("data/Ytr"+str(dataset)+".csv").to_numpy()[:,1]
        Ytr = Ytr.reshape((Ytr.shape[0],1))
        Ytr_reshaped = 2*Ytr.reshape(Ytr.shape[0])-1
        # Build Gram matrix:
        K_train = K(Xtr, Xtr, **kwargs)
        print("K_train successfully built")
        # Set problem for QP solver:
        ## Number of samples:
        n  = K_train.shape[0]
        ## Initial guess for mu:
        x0 = np.zeros(2*n)
        ## Matrix P:
        P = matrix((1/(2*lambd_SVM))*np.diag(Ytr_reshaped)@K_train@np.diag(Ytr_reshaped))
        ## Vector Q:
        q = -np.ones(n)
        q = matrix(q)
        ## Matrix G:
        G = np.zeros((2*n,n))
        G[:n] = -np.eye(n)
        G[n:] = np.eye(n)
        G = matrix(G)
        ## Vector h:
        h = np.zeros(2*n)
        h[n:]+= 1/n
        h = matrix(h)
        ## Call QP solver on dual SVM problem:
        resu_SVM = solvers.qp(P, q, G, h)
        mu = np.array(resu_SVM["x"])
        ## Recover primal solution:
        alpha = np.diag(Ytr_reshaped)@mu/(2*lambd_SVM)
        ## Build train prediction
        y_pred_SVM_raw = K_train@alpha
        y_pred_SVM = discriminate_vec(y_pred_SVM_raw)
        ## Show training accuracy:
        print('Training Accuracy for training set '+str(dataset)+' :',accuracy_score(y_pred_SVM, Ytr))
        # Test:
        ## Build Gram matrix:
        K_test = K(Xte, Xtr, **kwargs)
        print("K_test successfully built")
        ## Build test prediction:
        y_pred_SVM_test_raw = K_test@alpha
        y_pred_SVM_test = discriminate_vec(y_pred_SVM_test_raw)
        ## Build dataframe and save as .csv:
        start_index = dataset*1000
        dico = {"Id": range(start_index, start_index+Xte.shape[0]), "Bound": y_pred_SVM_test.reshape(Xte.shape[0]) }
        pred_SVM_test = pd.DataFrame(dico)
        pred_SVM_test.to_csv(name+".csv", index=False, sep=",", header=export_header, mode='a')
        export_header = False
        end_step = time.time()
        print("time on dataset {}: {}".format(dataset,np.round(end_step-start_step,2)))
    end_time = time.time()
    print("total time: {}".format(np.round(end_time-start_time,2)))

In [12]:
run_SVM(spectrum_kernel, "spectrum_kernel_SVM", lambd_SVM = 0.01, **{"k":10})

K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -2.0224e-01 -1.2030e+00  4e+03  6e+01  8e-16
 1: -2.0224e-01 -1.2024e+00  4e+01  7e-01  5e-16
 2: -2.0201e-01 -1.1532e+00  3e+00  3e-02  4e-16
 3: -1.9774e-01 -7.5575e-01  7e-01  3e-03  4e-16
 4: -1.9856e-01 -2.4293e-01  5e-02  1e-04  4e-16
 5: -1.9935e-01 -2.0662e-01  7e-03  2e-05  4e-16
 6: -1.9952e-01 -1.9993e-01  4e-04  9e-07  4e-16
 7: -1.9954e-01 -1.9956e-01  2e-05  4e-08  4e-16
 8: -1.9955e-01 -1.9955e-01  4e-07  6e-10  4e-16
 9: -1.9955e-01 -1.9955e-01  1e-08  7e-12  4e-16
Optimal solution found.
Training Accuracy for training set 0 : 0.999
K_test successfully built
time on dataset 0: 83.66
K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -2.0763e-01 -1.2083e+00  4e+03  6e+01  8e-16
 1: -2.0763e-01 -1.2078e+00  4e+01  7e-01  4e-16
 2: -2.0755e-01 -1.1614e+00  3e+00  3e-02  4e-16
 3: -2.0576e-01 -6.6502e-01  5e-01  4e-04  4e-16
 4: -2.0727e-01 -2.3088e-01  2e-02  2e-05  3

In [ ]:
run_SVM(mismatch_kernel, "mismatch_kernel_SVM", lambd_SVM = 0.01, **{"k":10})

K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.5398e-03 -1.0015e+00  4e+03  6e+01  2e-15
 1: -1.5398e-03 -1.0006e+00  4e+01  6e-01  2e-15
 2: -1.5266e-03 -9.0925e-01  1e+00  7e-03  2e-15
 3: -3.7110e-04 -8.2883e-02  9e-02  4e-04  2e-15
 4: -7.9306e-04 -1.7928e-02  2e-02  5e-05  2e-15
 5: -1.4790e-03 -2.2339e-03  8e-04  5e-07  2e-15
 6: -1.5269e-03 -1.6252e-03  1e-04  6e-08  2e-15
 7: -1.5298e-03 -1.5379e-03  8e-06  4e-09  2e-15
 8: -1.5302e-03 -1.5305e-03  3e-07  1e-10  2e-15
 9: -1.5302e-03 -1.5302e-03  1e-08  4e-12  2e-15
Optimal solution found.
Training Accuracy for training set 0 : 1.0
K_test successfully built
time on dataset 0: 2677.79
K_train successfully built
     pcost       dcost       gap    pres   dres
 0: -1.5607e-03 -1.0016e+00  4e+03  6e+01  2e-15
 1: -1.5607e-03 -1.0006e+00  4e+01  6e-01  2e-15
 2: -1.5473e-03 -9.0931e-01  1e+00  7e-03  2e-15
 3: -3.8692e-04 -9.9490e-02  1e-01  5e-04  2e-15
 4: -6.2410e-04 -1.5057e-02  2e-02  3e-05  2